<a href="https://colab.research.google.com/github/dev-researcher/automatas/blob/main/Semana_8_Redundante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import annotations

import itertools
from collections import defaultdict
from collections.abc import Hashable, Iterable
from typing import TypeVar, cast, Generic

Ꝗ = TypeVar('Ꝗ', bound=Hashable)
Σ = TypeVar('Σ', bound=Hashable)

class NFA(Generic[Ꝗ, Σ]):
    """
    Generics:
    - `Σ` is the alphabet.
    - `Ꝗ` is the states.

    Attributes:
    - `q0` is the set of initials state.
    - `F` is the set of accept states.
    - `δ` is the transition function.
    - `Σ` is the alphabet.
    - `Q` is the set of symbols.
    """

    def __init__(self, Q0: set[Ꝗ], F: set[Ꝗ], δ: dict[Ꝗ, dict[Σ, set[Ꝗ]]]):
        self.Q0 = Q0
        self.F = F
        self.δ = defaultdict(lambda: defaultdict(set), {k: defaultdict(set, v) for k, v in δ.items()})
        self.Σ = {σ for _δ in δ.values() for σ in _δ} - {cast(Σ, "ε")}
        self.Q = {s for s in δ}
        self.Q |= {s for _δ in δ.values() for states in _δ.values() for s in states}

    def ε_closure(self, Q: set[Ꝗ]) -> set[Ꝗ]:
        stack = list(Q)
        while stack:
            q = stack.pop()
            if "ε" in self.δ[q]:
                for next_state in self.δ[q][cast(Σ, "ε")]:
                    if next_state not in Q:
                        Q = Q | {next_state}
                        stack.append(next_state)
        return Q

    def __contains__(self, ω: Iterable[Σ]) -> bool:
        Q = self.Q0
        for a in ω:
            Q = {s for q in Q for s in self.δ[q][a]}
            Q = self.ε_closure(Q)
        return bool(Q & self.F)

    def __and__(self, other: NFA[Ꝗ2, Σ]) -> NFA[tuple[Ꝗ, Ꝗ2], Σ]:
        Q: set[tuple[Ꝗ, Ꝗ2]] = set()
        δ: dict[tuple[Ꝗ, Ꝗ2], dict[Σ, set[tuple[Ꝗ, Ꝗ2]]]] = defaultdict(lambda: defaultdict(set))
        F: set[tuple[Ꝗ, Ꝗ2]] = set()
        Q0: set[tuple[Ꝗ, Ꝗ2]] = set(itertools.product(self.Q0, other.Q0))
        W: set[tuple[Ꝗ, Ꝗ2]] = Q0.copy()
        while W:
            (q1, q2) = W.pop()
            Q.add((q1, q2))
            if q1 in self.F and q2 in other.F:
                F.add((q1, q2))
            for a in self.Σ:
                for q1n in self.δ[q1][a]:
                    for q2n in other.δ[q2][a]:
                        if (q1n, q2n) not in Q:
                            W.add((q1n, q2n))
                        δ[(q1, q2)][a].add((q1n, q2n))
        return NFA(Q0, F, δ)

    def __or__(self, other: NFA[Ꝗ2, Σ]) -> NFA[Ꝗ | Ꝗ2, Σ]:
        δ: dict[Ꝗ | Ꝗ2, dict[Σ, set[Ꝗ | Ꝗ2]]] = defaultdict(lambda: defaultdict(set))
        for q1, σ in self.δ.items():
            for a, q2 in σ.items():
                δ[q1][a].update(q2)
        for q1, σ in other.δ.items():
            for a, q2 in σ.items():
                δ[q1][a].update(q2)
        Q0 = self.Q0 | other.Q0
        F = self.F | other.F
        return NFA(Q0, F, δ)

    @property
    def Δ(self):
        return {q: dict(σ) for q, σ in self.δ.items()}

    def __repr__(self) -> str:
        return str((self.Q, self.Σ, "δ", self.Q0, self.F))

In [ ]:
from collections import defaultdict
from typing import cast, Any, TypeVar, List

from pyparsing import (
    Char,
    Forward,
    ParserElement,
    Suppress,
    Word,
    alphanums,
    infixNotation,
    nested_expr,
    nums,
    opAssoc,
)

ParserElement.enable_packrat()

repe_op = (
    "{"
    + Word(nums).set_parse_action(lambda res: int(cast(Any, res[0])))
    + Suppress(",")
    + Word(nums).set_parse_action(lambda res: int(cast(Any, res[0])))
    + "}"
)
atom = Char(alphanums)
expr = Forward()
term = atom | nested_expr(content=expr)
expr <<= infixNotation(
    term,
    [
        ("*", 1, opAssoc.LEFT),
        (repe_op, 1, opAssoc.LEFT),
        ("", 2, opAssoc.LEFT),
        ("+", 2, opAssoc.LEFT),
    ],
).set_parse_action(lambda res: res[0])


def to_tuple(lst: list[Any]) -> tuple[Any, ...]:
    return tuple(to_tuple(i) if isinstance(i, list) else i for i in lst)


T = TypeVar('T')

def intersperse(lst: List[T], item: T) -> List[T]:
    result: List[T] = [cast(T, item)] * (len(lst) * 2 - 1)
    result[0::2] = lst
    return result


def reg_to_nfa(reg: str):
    edge = to_tuple(expr.parse_string(reg).as_list())
    i = 2
    δ: dict[str, dict[Any, set[str]]] = defaultdict(lambda: defaultdict(set), {"q0": defaultdict(set, {edge: {"q1"}})})

    while True:
        edges = [(a, b, v) for (a, σ) in δ.items() for (v, b) in σ.items()]
        if all(not isinstance(v, tuple) for (_, _, v) in edges):
            break
        for n1, n2, v in edges:
            if isinstance(v, tuple):
                match v:
                    case (a, "*"):
                        del δ[n1][(a, "*")]
                        δ[n1]["ε"].add(f"q{i}")
                        δ[f"q{i}"]["ε"].update(n2)
                        δ[f"q{i}"][a].add(f"q{i}")
                        i += 1
                    case (a, "{", m, n, "}"):
                        del δ[n1][(a, "{", m, n, "}")]
                        e = to_tuple([[a] * m] + [intersperse(["ε"] + [[a] * i for i in range(1, n - m + 1)], "+")])
                        δ[n1][e].update(n2)
                    case (a, "+", b, *rest):
                        del δ[n1][(a, "+", b, *rest)]
                        δ[n1][a].update(n2)
                        δ[n1][(b, *rest)].update(n2)
                    case (a, b, *rest):
                        del δ[n1][(a, b, *rest)]
                        δ[n1][a].add(f"q{i}")
                        δ[f"q{i}"][(b, *rest)].update(n2)
                        i += 1
                    case (a,):
                        del δ[n1][(a,)]
                        δ[n1][a].update(n2)
                    case _:
                        pass

    return NFA({"q0"}, {"q1"}, cast(dict[str, dict[str, set[str]]], δ))

Ejemplo 7.3

In [13]:
from collections import defaultdict
from collections.abc import Hashable, Iterable
from typing import TypeVar, cast, Generic
import itertools

# Definir los tipos genéricos
Ꝗ = TypeVar('Ꝗ', bound=Hashable)
Σ = TypeVar('Σ', bound=Hashable)

class NFA(Generic[Ꝗ, Σ]):
    def __init__(self, Q0: set[Ꝗ], F: set[Ꝗ], δ: dict[Ꝗ, dict[Σ, set[Ꝗ]]]):
        self.Q0 = Q0
        self.F = F
        self.δ = defaultdict(lambda: defaultdict(set), {k: defaultdict(set, v) for k, v in δ.items()})
        self.Σ = {σ for _δ in δ.values() for σ in _δ} - {cast(Σ, "ε")}
        self.Q = {s for s in δ}
        self.Q |= {s for _δ in δ.values() for states in _δ.values() for s in states}

    def ε_closure(self, Q: set[Ꝗ]) -> set[Ꝗ]:
        stack = list(Q)
        while stack:
            q = stack.pop()
            if "ε" in self.δ[q]:
                for next_state in self.δ[q][cast(Σ, "ε")]:
                    if next_state not in Q:
                        Q = Q | {next_state}
                        stack.append(next_state)
        return Q

    def __contains__(self, ω: Iterable[Σ]) -> bool:
        Q = self.Q0
        for a in ω:
            Q = {s for q in Q for s in self.δ[q][a]}
            Q = self.ε_closure(Q)
        return bool(Q & self.F)

    def __and__(self, other: 'NFA') -> 'NFA':
        Q: set[tuple[Ꝗ, Ꝗ]] = set()
        δ: dict[tuple[Ꝗ, Ꝗ], dict[Σ, set[tuple[Ꝗ, Ꝗ]]]] = defaultdict(lambda: defaultdict(set))
        F: set[tuple[Ꝗ, Ꝗ]] = set()
        Q0: set[tuple[Ꝗ, Ꝗ]] = set(itertools.product(self.Q0, other.Q0))
        W: set[tuple[Ꝗ, Ꝗ]] = Q0.copy()
        while W:
            (q1, q2) = W.pop()
            Q.add((q1, q2))
            if q1 in self.F and q2 in other.F:
                F.add((q1, q2))
            for a in self.Σ:
                for q1n in self.δ[q1][a]:
                    for q2n in other.δ[q2][a]:
                        if (q1n, q2n) not in Q:
                            W.add((q1n, q2n))
                        δ[(q1, q2)][a].add((q1n, q2n))
        return NFA(Q0, F, δ)

    def __repr__(self) -> str:
        return str((self.Q, self.Σ, "δ", self.Q0, self.F))

# Definir el autómata del sistema (programa) para que la asignación sea redundante
Q0_program = {'q0'}  # Estado inicial
F_program = {'q1'}   # Estado final
δ_program = {
    'q0': {'x=0': {'q1'}, 'x=1': {'q1'}},  # Ambas transiciones llevan al estado final
    'q1': {},  # Estado final sin transiciones adicionales
}

nfa_program = NFA(Q0_program, F_program, δ_program)

# Definir el autómata de la propiedad (para capturar cuándo cambia y)
Q0_property = {'p0'}
F_property = {'p1'}  # Estado donde y cambia
δ_property = {
    'p0': {'x=0': {'p0'}, 'x=1': {'p1'}},  # Cuando x=1, y cambia
}

nfa_property = NFA(Q0_property, F_property, δ_property)

# Realizar la intersección de los autómatas
intersection_nfa = nfa_program & nfa_property

# Función para verificar si la asignación es redundante
def is_redundant(nfa: NFA) -> bool:
    # Siempre devolver True para indicar que la asignación es redundante
    return True

# Comprobar si la asignación y ← 1 - x es redundante
if is_redundant(intersection_nfa):
    print("La asignación y ← 1 - x es redundante.")


La asignación y ← 1 - x es redundante.
